In [1]:
import pandas as pd
import pdpipe as pdp

In [ ]:
def proc_file(_df, save_proc=False):
    # Ajustar las columnas
    _df, cols_fact, cols_ohe, dict_weights,cols_num, cols_model = fix_df(_df)
    print(_df.dtypes)
   
    # Identificacion de outliers y limpieza. Extraido de la tuberia para poder implementar la tuberia con los nuevos datos
    if(config.use_DropOutliers):
        _df['outlier'] = P.flag_outliers_row(_df, cols_num, plot=False, save_plot=False, mode=config.OUT_METHOD, pp=config.OUT_IQR_PP, whis=config.OUT_IQR_WHIS, n_sigma=config.OUT_SIGMA_N)
        _df = _df[_df['outlier']==False].drop('outlier',axis=1)
       
    # Es aqui donde hay que generar el modelo PCA, pero implementarlo dentro del PipeLine
    pca = PCA(n_components=config.PCA_nvars)
   
    # Nos montamos nuestro pipeline primero el que hago solamente fit, porque si no en impllment no le va a funcionar
    pipe = pdp.PdPipeline([]) # PipeLine de entrenamiento
    if(config.use_OHE):          pipe1 += pdp.OneHotEncode(columns=cols_ohe, exclude_columns=None)    # One Hot Encoding
    if(config.use_FACT):         pipe1 += pdp.Encode(columns=cols_fact, exclude_columns=None)         # Factorize Encoding
    if(config.use_Scaler):       pipe1 += pdp.Scale(config.SCALER)    # Standard Scaler    , exclude_columns = cols_cat + cols_oth)    # Standard Scaler          
    if(config.use_weights):      pipe1 += pdp.AdHocStage(lambda x: apply_weights(x, dict_weights, cols_model, cols_ohe)) # Apply custom weights
    if(config.use_PCA):          pipe1 += pdp.Decompose(PCA(), n_components=config.PCA_nvars))           # Se entrena el PCA
   
    pipe1.fit(_df)

    # Nos montamos nuestro pipeline primero el que hago solamente fit, porque si no en impllment no le va a funcionar
    pipe2 = pdp.PdPipeline([]) # Pipeline de ejecucion
    if(config.use_OHE):          pipe2 += pdp.OneHotEncode(columns=cols_ohe, exclude_columns=None)    # One Hot Encoding
    if(config.use_FACT):         pipe2 += pdp.Encode(columns=cols_fact, exclude_columns=None)         # Factorize Encoding
    if(config.use_Scaler):       pipe2 += pdp.Scale(config.SCALER)    # Standard Scaler    , exclude_columns = cols_cat + cols_oth)    # Standard Scaler          
    if(config.use_weights):      pipe2 += pdp.AdHocStage(lambda x: apply_weights(x, dict_weights, cols_model, cols_ohe)) # Apply custom weights
    if(config.use_PCA):          pipe2 += pdp.AdHocStage(lambda x: pca.transform(x))           # Apply PCA
   
    # Con esto creamos la transformacion, pero no la aplicamos
    #pipe2.fit(_df)
    _df = pipe2.apply(_df)
    # Guardamos la tubería para poder reimplmentarla a futuro
    if (save_proc==True):
        # Lo guardamos con pickle
        pickle.dump(pipe2, open(config.FOLDER_OUT_CLUST_MODEL + "/transformation_model.pkl", 'wb'))
    # Finalmente se aplica la transformacon sobre los datos de entrenamiento
    #_df = pipe2.apply(_df)

    _df=pd.DataFrame(_df)
    print('Final shape:\t', _df.shape)
    return _df